**Set environment**

In [2]:
### set env
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc



**TEST**

In [3]:
ls ${FD_RES}/region

annotation_cCREs  annotation_remap2022  KS91_K562_astarrseq_peak_macs


In [4]:
ls ${FD_RES}/region/KS91_K562_astarrseq_peak_macs

KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.ccre_all.bed.gz
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.nuc.bed.gz


In [5]:
ls ${FD_RES}/region/annotation_remap2022

remap2022_K562_all_macs2_hg38_v1_0.bed.gz
remap2022_K562_crm_macs2_hg38_v1_0.bed.gz
remap2022_K562_nr_macs2_hg38_v1_0.bed.gz


## Annotation with CCRE

In [11]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/annotation_astarr_peak_macs_input_remap.txt \
    --array 0 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh

### set input and output
FD_BED=${FD_RES}/region/KS91_K562_astarrseq_peak_macs
FN_BED=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
FP_BED=${FD_BED}/${FN_BED}

FD_ANN=${FD_RES}/region/annotation_remap2022
FN_ANN=remap2022_K562_nr_macs2_hg38_v1_0.bed.gz
FP_ANN=${FD_ANN}/${FN_ANN}

FD_OUT=${FD_RES}/region/KS91_K562_astarrseq_peak_macs
FN_OUT=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.remap2022_nr.bed.gz
FP_OUT=${FD_OUT}/${FN_OUT}

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### show I/O file
echo "Input: " ${FP_BED}
echo
echo "show first few lines of input"
cat ${FP_BED} | head -5
echo
echo "Input: " ${FP_ANN}
echo
echo "show first few lines of input"
zcat ${FP_ANN} | head -5
echo

### init: create output folder if not exist
mkdir -p ${FD_OUT}

### annotation using intersect
bedtools intersect -a ${FP_BED} -b ${FP_ANN} -wo -F 1.0 | gzip -c > ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29160140


**CHECK**

In [12]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_remap.txt

Hostname:           x1-01-3.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         10-17-22+10:32:28

Input:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_astarrseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/annotation_remap2022/remap2022_K562_nr_macs2_hg38_v1_0.bed.gz

show first few lines of input
chr1	9899	10349	EP400:K-562	1	.	10089	10090	102,132,184
chr1	9901	10232	NCOA1:K-562	1	.	10078	10079	56,224,224
chr1	9902	10328	ZBTB40:MCF-7,Hep-G2,K-562,GM12878	4	.	10093	10094	73,149,46
chr1	9905	10132	AFF1:K-562	1	.	10088	10089	11,34,77
chr1	9905	10288	ZNF184:K-562	1	.	10104	10105	157,198,124


Output:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_astarrseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input